In [6]:
%pip install pandas matplotlib -q
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import requests


[notice] A new release of pip is available: 23.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Park id for thorpe park
park_id = 2

# Read into pandas df from sqlite fromm root directory, then date/queue_data.db
# Read the park_info and queue_data tables
conn = sqlite3.connect('../data/queue_data.db')
park_info = pd.read_sql_query(f"SELECT * FROM park_info WHERE park_id = {park_id}", conn)
queue_data = pd.read_sql_query("SELECT * FROM queue_data", conn)